In [1]:
#Following libraries are required to be installed before proceeding further
!pip install -U langchain langchainhub langchain_community pypdf sentence-transformers faiss-cpu langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.7/268.7 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14

## Downloading PDF Docuemnt From URL

## Loading PDF Documents Using LangChain Document Loader
LangChain's DirectoryLoader implements functionality for reading files from disk into LangChain Document objects.

In [2]:
# loading documnet
from langchain_community.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("Desktop/My Githup/Deep-Learning-Projects/RAG/docs")
docs = loader.load()
print(f'{len(docs)} page document loaded.')

2 page document loaded.


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

'''
chunk_size=1000: Each chunk will have up to 1000 characters.
chunk_overlap=200: Each chunk will overlap with the previous chunk by 200 characters. 
This overlap helps maintain context across chunks, ensuring that the split doesn't cut off important context information.
'''
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
splits = text_splitter.split_documents(docs)
print(f'{len(splits)} chunks created.')

97 chunks created.


## Setting Up Embeddings Model with LangChain
Set up the embeddings model using LangChain and a default HuggingFace model.

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

# Initialize the HuggingFace embeddings model
embeddings_model = HuggingFaceEmbeddings()

2024-11-15 23:13:49.018154: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 23:13:49.018305: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 23:13:49.198759: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings_model)

## Performing Similarity Search

In [6]:
# it allows you to record messages (logs) during program execution
import logging

# configuring the root logger (the highest-level logger in the logging hierarchy) to default settings
logging.basicConfig()

# retrieving a logger object named “langchain.retrievers.” which allows you to organize and manage log messages
logging.getLogger("langchain.retrievers").setLevel(logging.INFO)

# performing a similarity search using the given query (“What is software architecture?”) and returns the top 5 results.
vectorstore.similarity_search("What is software architecture?", top_k=4)

[Document(metadata={'source': '/kaggle/input/my-datacv/Data Scientist Enejan.pdf', 'page': 1}, page_content='Management: Agile methodologies, SDLC, Google Sheets, MS Excel Soft Skills: Analytical'),
 Document(metadata={'source': '/kaggle/input/my-datacv/Data Scientist Enejan.pdf', 'page': 0}, page_content='reasoning and analytical skills.  • Utilized Excel for data tracking and progress analysis,'),
 Document(metadata={'source': '/kaggle/input/my-datacv/Data Scientist Enejan.pdf', 'page': 1}, page_content='Deep Learning (CNN, ANN, LSTM, GRU), NLP, Computer Vision; Tools & Frameworks: Jupyter Notebook,'),
 Document(metadata={'source': '/kaggle/input/my-datacv/Data Scientist Enejan.pdf', 'page': 1}, page_content='– Data Analytics Essentials, Data Analysis Program, Data Science Program ProjectsMachine Learning')]

## Joining Strings from Multiple Documents

In [7]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

'''
The function joins these page_content strings together using double newline characters ("\n\n").
This function is designed to concatenate the page content from multiple documents, 
ensuring that each page’s content is separated by double newlines. 
It’s useful for formatting or further processing document data.
'''


'\nThe function joins these page_content strings together using double newline characters ("\n\n").\nThis function is designed to concatenate the page content from multiple documents, \nensuring that each page’s content is separated by double newlines. \nIt’s useful for formatting or further processing document data.\n'

## Loading pre-trained LLM on Hugging Face

In [8]:
# first getting the API Key stored in Kaggle Secrets
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("api-key")

In [9]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    huggingfacehub_api_token=secret_value_0,# your HF token here
    max_new_tokens=512
)

chat = ChatHuggingFace(llm=llm, verbose=True)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## Retrieving the relevant information

In [10]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Retrieve and generate using the relevant information from the document.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chat
    | StrOutputParser() # Converts the language model output into a string format
)


/opt/conda/lib/python3.10/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [11]:
rag_chain.invoke("Who is data scintist")

' A data scientist is a professional who uses scientific methods, processes, algorithms, and systems to extract knowledge and insights from structured and unstructured data. Based on the context provided, Enejan Hayytjanova is a Junior Data Scientist with experience in machine learning, data analysis, and deep learning, among other related skills.'